In [1]:
import mne
from glob import glob
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session

In [2]:
all_files_path=glob('../dataset/data/*.edf')
print(len(all_files_path))

162


In [3]:
healthy_file_path=[i for i in all_files_path if  'H' in i.split('\\')[1]]
patient_file_path=[i for i in all_files_path if  'M' in i.split('\\')[1]]
print(len(healthy_file_path),len(patient_file_path))

76 86


In [4]:
def read_data(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw.pick_types(meg=False, eeg=True, eog=False, ecg=False) # Selecting EEG, EOG and ECG channels
    # Select a specific channel
    channel_to_keep = ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE', 'EEG A2-A1']  
    # Replace with the name of the channel you want to keep
    raw.pick_channels(channel_to_keep)
    raw.set_eeg_reference()
    raw.filter(l_freq=30,h_freq=100)#1-4=delta, 4-8=theta, 8-12=alpha, 12-30=beta, 30-100=gamma
    epochs=mne.make_fixed_length_epochs(raw,duration=15,overlap=1)
    epochs=epochs.get_data()
    scaler = StandardScaler()
    data = scaler.fit_transform(epochs.reshape(-1,epochs.shape[-1])).reshape(epochs.shape)

    result=[]
    for p in range(len(data)):
        result_2d = []
        for q in range(len(data[p])):
            
            arr=data[p][q]

            window_size = 15
            i = 0
            # Initialize an empty list to store moving averages
            moving_averages = []
            
            # Loop through the array to consider
            # every window of size 3
            while i < len(arr) - window_size + 1:
            
                # Store elements from i to i+window_size
                # in list to get the current window
                window = arr[i : i + window_size]
            
                # Calculate the average of current window
                window_average = round(sum(window) / window_size, 2)
                
                # Store the average of current
                # window in moving average list
                moving_averages.append(window_average)
                
                # Shift window to right by one position
                i += 1
            result_2d.append(moving_averages)
        result.append(result_2d)
    return result

In [5]:
%%capture
control_epochs_array = [read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]


In [ ]:
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len(i)*[1] for i in patients_epochs_array]

data_list=control_epochs_array+patients_epochs_array
label_list=control_epochs_labels+patients_epochs_labels
# groups_list=[[i]*len(j) for i, j in enumerate(data_list)]

data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
data_array=np.moveaxis(data_array,1,2)

In [1]:
# Assuming 'results' is a list containing results for all 76 files
for i, result in enumerate(control_epochs_array):
    print(i)

NameError: name 'control_epochs_array' is not defined

In [ ]:
# Assuming 'moving_averages' is the list containing the moving averages for a single EEG file
def plot_moving_averages(moving_averages):
    num_windows = len(moving_averages)  # Number of windows
    
    plt.figure(figsize=(10, 5))
    
    for i in range(num_windows):
        plt.plot(moving_averages[i], label=f'Window {i+1}')
    
    plt.title('Moving Averages Plot')
    plt.xlabel('Time')
    plt.ylabel('Moving Average')
    plt.legend()
    plt.show()

# Call the plotting function with the list of moving averages
plot_moving_averages(moving_averages)

In [20]:
control_epochs_array[0]

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
import matplotlib.pyplot as plt

def plot_data_after_moving_average(result, file_name):
    num_channels = len(result[0])  # Number of channels
    num_windows = len(result)      # Number of windows
    
    fig, axs = plt.subplots(num_channels, 1, figsize=(10, 5*num_channels))
    
    for i in range(num_channels):
        axs[i].set_title(f'Channel {i+1}')
        axs[i].set_xlabel('Time')
        axs[i].set_ylabel('Moving Average')
        
        # Plot moving averages for each window in the channel
        for j in range(num_windows):
            axs[i].plot(result[j][i], label=f'Window {j+1}')
        
        axs[i].legend()
    
    plt.tight_layout()
    plt.savefig(f'{file_name}.pdf')  # Save the figure to PDF
    plt.close()  # Close the figure to release memory

# Assuming 'results' is a list containing results for all 76 files
for i, result in enumerate(control_epochs_array):
    file_name = f'figures/Figure_{i+1}'  # Generate a file name
    plot_data_after_moving_average(result, file_name)

In [ ]:
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(3840,20)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11
    
    model.compile('adam',loss='binary_crossentropy',metrics=['Accuracy', 'Precision', 'Recall','AUC'])
    return model

model=cnnmodel()
model.summary()

In [ ]:
print(data.shape)

In [ ]:
print(data[0][0])

In [ ]:
result=[]
for p in range(len(data)):
    result_2d = []
    for q in range(len(data[p])):
        
        arr=data[p][q]

        window_size = 15
        i = 0
        # Initialize an empty list to store moving averages
        moving_averages = []
        
        # Loop through the array to consider
        # every window of size 3
        while i < len(arr) - window_size + 1:
        
            # Store elements from i to i+window_size
            # in list to get the current window
            window = arr[i : i + window_size]
        
            # Calculate the average of current window
            window_average = round(sum(window) / window_size, 2)
            
            # Store the average of current
            # window in moving average list
            moving_averages.append(window_average)
            
            # Shift window to right by one position
            i += 1
        result_2d.append(moving_averages)
    result.append(result_2d)

In [ ]:
len(result[0][0])

In [ ]:
smoothed_data

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(data[0,0], label='Original Data')
plt.plot(smoothed_data[0,0], label='Moving Average', linestyle='--')
plt.xlabel('Time')
plt.ylabel('EEG Amplitude')
plt.title('Original Data vs. Moving Average')
plt.legend()
plt.show()